## Descripción del proyecto 

Los clientes de Beta Bank se están yendo, cada mes, poco a poco. Los banqueros descubrieron que es más barato salvar a los clientes existentes que atraer nuevos.

Necesitamos predecir si un cliente dejará el banco pronto. Tú tienes los datos sobre el comportamiento pasado de los clientes y la terminación de contratos con el banco.

## Descripción de los datos

Características

- RowNumber: índice de cadena de datos
- CustomerId: identificador de cliente único
- Surname: apellido
- CreditScore: valor de crédito
- Geography: país de residencia
- Gender: sexo
- Age: edad
- Tenure: período durante el cual ha madurado el depósito a plazo fijo de un cliente (años)
- Balance: saldo de la cuenta
- NumOfProducts: número de productos bancarios utilizados por el cliente
- HasCrCard: el cliente tiene una tarjeta de crédito (1 - sí; 0 - no)
- IsActiveMember: actividad del cliente (1 - sí; 0 - no)
- EstimatedSalary: salario estimado

Objetivo

- Exited: El cliente se ha ido (1 - sí; 0 - no)

## Análisis exploratorio de datos

In [ ]:
# Cargamos librerías a utilizar
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.utils import shuffle

In [ ]:
# Cargamos nuestro df y observamos información importante
data = pd.read_csv('/datasets/Churn.csv')

data.info()
data.head()

In [ ]:
data['Tenure'] = data['Tenure'].fillna(0.0)  # Reemplazar NaN por 0

## Segmentación de datos 

In [ ]:
#Se utiliza la codificacion ohe para poder transformar las caracteristicas categoricas en numericas
data_ohe = pd.get_dummies(data, columns=['Geography', 'Gender'], drop_first=True)
target = data_ohe['Exited']
features = data_ohe.drop(['Exited', 'RowNumber', 'CustomerId', 'Surname'], axis=1)

#Se divide el conjunto de datos
features_train, features_temp, target_train, target_temp = train_test_split(features, target, test_size=0.4, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(features, target, test_size=0.5, random_state=12345) 


## Prueba inicial

In [ ]:
model = LogisticRegression (random_state=12345, solver='liblinear')
model.fit(features_train, target_train)

print('¡Entrenado!')

In [ ]:
# Predecir en el conjunto de prueba
predicted_valid = model.predict(features_valid)

print(accuracy_score (target_valid ,predicted_valid))
print('F1:', f1_score(target_valid, predicted_valid)) #Se verifica el valor F1
print ('auc_roc:', roc_auc_score(target_valid, model.predict_proba(features_valid)[:, 1])
)

**Se tiene un valor F1 sumamente bajo, por lo que se concluye que este modelo no sirve, aún asi se comparaara este modelo contra uno constante**

## Prueba de consistencia

In [ ]:
target_pred_constant = pd.Series([0] * len(target)) #Se crea un modelo constante 

print(accuracy_score(target, target_pred_constant))

**Se tiene el mismo resultado tanto en el modelo constante como en el modelo supuestamente entrenado anteriormente, por lo que se sabe que el modelo que se entreno no es confiable.**

## Corrección de desequilibrio de datos y eleccion de mejor modelo

In [ ]:
model_balanced = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced') #se utiliza el argumento class_weight para ajustar el peso de las clases
#Se entrena un nuevo modelo balanceado
model_balanced.fit(features_train, target_train)
predicted_valid_b = model_balanced.predict(features_valid)

print('F1:', f1_score(target_valid, predicted_valid_b)) #Se calcula nuevamente el valor F1
print ('auc_roc:', roc_auc_score(target_valid, model_balanced.predict_proba(features_valid)[:, 1])
)

**Se observa un amplia mejora en nuestro modelo, es mas confiable que el anterior, sin embargo, aún no es suficiente**

In [ ]:
#se crea una funcion para aplicar la tecnica de submuestreo
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)]
        + [features_ones]
    )
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)]
        + [target_ones]
    )

    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345
    )

    return features_downsampled, target_downsampled


features_downsampled, target_downsampled = downsample(
    features_train, target_train, 0.1
)

#Se entrena un tercer modelo
model_c = LogisticRegression (random_state=12345, solver='liblinear', class_weight='balanced')
model_c.fit(features_downsampled, target_downsampled)
predicted_valid_c = model_c.predict(features_valid)# < escribe el código aquí >

print('F1:', f1_score(target_valid, predicted_valid_c))
print ('auc_roc:', roc_auc_score(target_valid, model_c.predict_proba(features_valid)[:, 1])
)

**En este caso se observa que se tiene un valor menor de F1 en comparacion al metodo anterior, por lo que se sabe que no se seguira usando el metodo de submuetreo.**

**Dado que entrenando el modelo con regresión logistica no se obtuvieron resultados satisfactorios, ahora se procedera a entrenar el modelo con un arbol de decision.**

In [ ]:
for depth in range(1, 11):  # selecciona el rango del hiperparámetro
    model_d = DecisionTreeClassifier(random_state = 54321, max_depth=depth, class_weight='balanced')
    model_d.fit(features_train, target_train)
    predicted_valid_d = model_d.predict(features_valid)
    test_predictions_d = model_d.predict(features_test)
    
    print ('conjunto de validacion', depth)
    print ('f1:', f1_score(target_valid, predicted_valid_d))
    print ('auc_roc:', roc_auc_score(target_valid, model_d.predict_proba(features_valid)[:, 1]))
    print ('conjunto de prueba', depth)
    print ('f1:', f1_score(target_test, test_predictions_d))
    print ('auc_roc:', roc_auc_score(target_test, model_d.predict_proba(features_test)[:, 1]))
    print()

**Se observa que cuando el valor de max_depth es igual a 5, el F1-score en el conjunto de prueba alcanza su valor máximo. Además, en el conjunto de entrenamiento se obtiene una puntuación similar, lo que indica que hasta ahora este modelo es el mejor entrenado.**

**Este modelo presenta un buen balance tanto con el conjunto de entrenamiento como con el de prueba, por lo que hasta el momento, es el mejor modelo que se ha entrenado**

**Se entrenara el modelo con bosque aleatorio**

In [ ]:
best_f1 = 0
best_est = 0
for est in range(1, 51): # selecciona el rango del hiperparámetro
    model_e = RandomForestClassifier(random_state=54321, n_estimators=est, class_weight='balanced') # configura el número de árboles
    model_e.fit(features_train, target_train) # entrena el modelo en el conjunto de entrenamiento
    
    predicted_valid_e = model_e.predict(features_valid)
    test_predictions_e = model_e.predict(features_test)
    
    f1_valid = f1_score(target_valid, predicted_valid_e)
    f1_test = f1_score(target_test, test_predictions_e)
    
    if f1_test > best_f1:
        best_f1 = f1_test# guarda la mejor puntuación de accuracy en el conjunto de validación
        best_est = est# guarda el número de estimadores que corresponden a la mejor puntuación de exactitud

        print("Para n_estimators = {}: F1-score (validación): {}, F1-score (prueba): {}".format(est, f1_valid, f1_test))

print("El mejor modelo tiene n_estimators = {} con un F1-score en validación de: {}".format(best_est, best_f1))

**Este modelo presenta variaciones bastante grandes de un conjunto de datos a otro, por lo cual este modelo se descarta.**

**Se concluye que el mejor modelo para este caso es el arbol de decisiones**

## Prueba final

In [ ]:
model_d = DecisionTreeClassifier(random_state = 54321, max_depth=5, class_weight='balanced')
model_d.fit(features_train, target_train)
test_predictions_d = model_d.predict(features_test)

print ('f1:', f1_score(target_test, test_predictions_d))
print ('auc_roc:', roc_auc_score(target_test, model_d.predict_proba(features_test)[:, 1]))

**Se obtiene el mejor resultado posible con el modelo que mostro mejor desempeño.**